In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
import re
from nltk.tokenize import word_tokenize
import string
from sklearn.model_selection import train_test_split
from datetime import datetime, date

# Import Data

In [24]:
data = pd.read_csv("data_twitter.csv")

In [25]:
data

,Unnamed: 0,created_at,date,time,username,tweet
0,0,2021-11-21 14:30:01 SE Asia Standard Time,2021-11-21,14:30:01,nowbitcoinprice,"Nov 21, 2021 07:29:00 UTC | 59,158.80$ | 52,41..."
1,1,2021-11-21 14:22:26 SE Asia Standard Time,2021-11-21,14:22:26,bitcoingraph,[BTCUSDT - H4] #BTC #BTCUSDT #BINANCE #BTCUSD ...
2,2,2021-11-21 14:17:14 SE Asia Standard Time,2021-11-21,14:17:14,shanaz3197,@tunefmofficial $JAM $HBAR #NFTs #Crypto #TUNEFM
3,3,2021-11-21 14:10:38 SE Asia Standard Time,2021-11-21,14:10:38,mdshipo22750084,@tunefmofficial $JAM $HBAR #NFTs #Crypto #TUNEFM
4,4,2021-11-21 14:07:27 SE Asia Standard Time,2021-11-21,14:07:27,khaiyaraya,"First Meeting, Crypto Trading. Kepada yang i..."
...,...,...,...,...,...,...
8472,8472,2021-11-12 19:15:09 SE Asia Standard Time,2021-11-12,19:15:09,tribunpontianak,Alasan MUI Haramkan Kripto #BeritaTerkini #ber...
8473,8473,2021-11-12 19:10:33 SE Asia Standard Time,2021-11-12,19:10:33,tribunpontianak,Apa itu Kripto ? Sejarahnya ? #BeritaTerkini #...
8474,8474,2021-11-12 15:00:23 SE Asia Standard Time,2021-11-12,15:00:23,suhukripto,"Kali ini FUD datang dari dalam negri guys, Hmm..."
8475,8475,2021-11-12 14:00:21 SE Asia Standard Time,2021-11-12,14:00:21,dialektikakng,"Hasil Ijtima Ulama ke-VII, Komisi Fatwa MUI Me..."


In [26]:
data.isna().sum()

Unnamed: 0    0
created_at    0
date          0
time          0
username      0
tweet         0
dtype: int64

In [28]:
data = data.drop_duplicates(subset=['tweet'])
data

,Unnamed: 0,created_at,date,time,username,tweet
0,0,2021-11-21 14:30:01 SE Asia Standard Time,2021-11-21,14:30:01,nowbitcoinprice,"Nov 21, 2021 07:29:00 UTC | 59,158.80$ | 52,41..."
1,1,2021-11-21 14:22:26 SE Asia Standard Time,2021-11-21,14:22:26,bitcoingraph,[BTCUSDT - H4] #BTC #BTCUSDT #BINANCE #BTCUSD ...
2,2,2021-11-21 14:17:14 SE Asia Standard Time,2021-11-21,14:17:14,shanaz3197,@tunefmofficial $JAM $HBAR #NFTs #Crypto #TUNEFM
4,4,2021-11-21 14:07:27 SE Asia Standard Time,2021-11-21,14:07:27,khaiyaraya,"First Meeting, Crypto Trading. Kepada yang i..."
5,5,2021-11-21 14:00:01 SE Asia Standard Time,2021-11-21,14:00:01,nowbitcoinprice,"Nov 21, 2021 06:59:00 UTC | 59,278.70$ | 52,51..."
...,...,...,...,...,...,...
5784,5784,2021-11-21 14:49:42 SE Asia Standard Time,2021-11-21,14:49:42,stevenjhsiahaan,"Halo! Sekarang, kamu bisa investasi saham luar..."
5785,5785,2021-11-21 14:45:28 SE Asia Standard Time,2021-11-21,14:45:28,samuelcendikia1,"Halo! Sekarang, kamu bisa investasi saham luar..."
5786,5786,2021-11-21 14:39:45 SE Asia Standard Time,2021-11-21,14:39:45,saturnusgo,@binnybunny96 @BigAlphaID Kripto haram mbak! ...
5787,5787,2021-11-21 14:35:25 SE Asia Standard Time,2021-11-21,14:35:25,muhamad25615484,"Halo! Sekarang, kamu bisa investasi saham luar..."


# Preprocessing

**Emoji detection**

In [31]:
import string
punct = set(string.punctuation)
print(punct)

{';', '=', '-', ',', ']', '\\', '(', '$', '?', '*', '`', '[', '<', "'", ')', '}', ':', '@', '!', '.', '~', '#', '|', '+', '_', '&', '>', '%', '/', '^', '"', '{'}


In [32]:
emoji = set()
for s in data['tweet'].fillna('').astype(str):
    for c in s:
        if c.isdigit() or c.isalpha() or c.isalnum() or c.isspace() or c in punct:
            continue
        emoji.add(c)
        
print(''.join(emoji))

🧩🤔😭😳🌹😊👹€™£😬🙇🙈🟠✨🤬💹🚩😝😡🥶✌✔🟥💪🍀✊🔔😞😹⬅🎗🗿🏃🤙👎🤫🎉⌛💎’👾🤖🤩🦊…😜🤌🚗❗😶🥴🌤😵📰📍🤨💲🍇👂🥺🤪🔴😎😉🛡🇮👍🇺🤯🌟🤭🎙🌍❤–‍🆓💨♂🐕😑🏼🗓👊😃°😄😇⚡🥱😔📌©✅₹⏰💣🧭🪐🔥🕯😤💕🚀💻😒👇🤡🧓😐🎣🙌🔗☑🐉😘😛🖤😙💯💥❣🤝🪙🥳🙅💫🗞🥷️😁▶😀🧜🐇»🤑🍷🧐👏😩🔸😅🔵🤞🤷📣😖🏝🥲🤣🍯💞⁦⁩👋🌧♀🤦➡🚜🙄🔨🔹🌙🖥—😆🏿🇲💵🧠😌📲😢😂🏻👀😍“🧎💀🇩📈😱🆗🚨🙃💰⚓‘❌✍🌚🥰🙏☺🥇🐖🦆😥👉👌•☕🙂😣”👆🔊🙀💡


**Dowload Stopword**

#Data Cleaning

Data Cleaning adalah proses mendeteksi dan memperbaiki catatan yang korup, tidak akurat,atau tidak relevan dari data, kemudian mengganti, memodifikasi, atau menghapus data yang kotor tersebut. 

In [33]:
data['tweet']=data['tweet'].apply(str)

<ipython-input-33-a30139c8769e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet']=data['tweet'].apply(str)


**Remove URL**

In [34]:
example="New competition launched :https://www.kaggle.com/c/nlp-getting-started"

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

remove_URL(example)

'New competition launched :'

In [35]:
data['tweet'] = data['tweet'].apply(lambda x : remove_URL(x))

<ipython-input-35-d844ca80495f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet'] = data['tweet'].apply(lambda x : remove_URL(x))


**Remove Emoji**

In [36]:
import string
punct = set(string.punctuation)
print(punct)

{';', '=', '-', ',', ']', '\\', '(', '$', '?', '*', '`', '[', '<', "'", ')', '}', ':', '@', '!', '.', '~', '#', '|', '+', '_', '&', '>', '%', '/', '^', '"', '{'}


In [37]:
emoji = set()
symbol = 'ðŸ˜â\x81£â\x81£'
for s in data['tweet'].fillna('').astype(str):
    for c in s:
        if c.isdigit() or c.isalpha() or c.isalnum() or c.isspace() or c in punct:
            continue
        emoji.add(c)

for s in symbol:
  emoji.add(s)

print(''.join(emoji))

🧩🤔😭😳🌹😊👹€™£😬🙇🙈🟠✨🤬💹🚩😝😡🥶✌✔🟥💪🍀✊🔔😞😹⬅🎗🗿🏃🤙👎🤫🎉ð⌛💎’👾🤖🤩🦊…😜🤌🚗❗😶🥴🌤😵📰📍🤨💲🍇👂🥺🤪🔴😎😉🛡🇮👍🇺🤯🌟🤭🎙🌍❤–‍🆓💨♂🐕😑🏼🗓👊😃°😄😇⚡🥱😔📌©✅₹⏰â💣🧭🪐🔥🕯😤💕🚀💻😒👇🤡🧓😐🎣🙌🔗˜☑🐉😘😛🖤😙💯💥❣🤝🪙🥳🙅💫🗞🥷️😁▶😀🧜🐇»Ÿ🤑🍷🧐👏😩🔸😅🔵🤞🤷📣😖🏝🥲🤣🍯💞⁦⁩👋🌧♀🤦➡🚜🙄🔨🔹🌙🖥—😆🏿🇲💵🧠😌📲😢😂🏻👀😍“🧎💀🇩📈😱🆗🚨🙃💰⚓‘❌✍🌚🥰🙏☺🥇🐖🦆😥👉👌•☕🙂😣”👆🔊🙀💡


In [39]:
def remove_emoji(text):
  temp = ''
  for c in text:
    if c not in emoji:
      temp += c
  
  return temp
  
remove_emoji('Semainya satu, tumbuhnya dua.. ðŸ˜…â£â£ Kalo dijual perpohon bisa menang banyak ini ðŸ˜ðŸ˜â£.â£.â£.')

'Semainya satu, tumbuhnya dua..  Kalo dijual perpohon bisa menang banyak ini ...'

In [40]:
data['tweet'] = data['tweet'].apply(lambda x : remove_emoji(x))

<ipython-input-40-45cfc3f81f0e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet'] = data['tweet'].apply(lambda x : remove_emoji(x))


**Remove Hastag**

In [41]:
#https://ichi.pro/id/membersihkan-data-teks-dengan-python-200778039524214

def remove_hastag(x):
  x = re.sub(r'#\S+', ' ', x)
  return x

print(remove_hastag('kalo dijual perpohon...#simpleplanthidroponik #simpleplantbyyuka #simpleplant #selada'))

kalo dijual perpohon...       


In [42]:
data['tweet'] = data['tweet'].apply(lambda x : remove_hastag(x))

<ipython-input-42-63a0896b0156>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet'] = data['tweet'].apply(lambda x : remove_hastag(x))


**Remove Punctuation**

In [43]:
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

example="I am a #king"
print(remove_punct(example))

I am a king


In [44]:
data['tweet'] = data['tweet'].apply(lambda x : remove_punct(x))

<ipython-input-44-375d9d198427>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet'] = data['tweet'].apply(lambda x : remove_punct(x))


**Make Lower**

In [45]:
data['tweet'] = data['tweet'].apply(lambda x : x.lower())

<ipython-input-45-86a8dbfe4695>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet'] = data['tweet'].apply(lambda x : x.lower())


In [46]:
data

,Unnamed: 0,created_at,date,time,username,tweet
0,0,2021-11-21 14:30:01 SE Asia Standard Time,2021-11-21,14:30:01,nowbitcoinprice,nov 21 2021 072900 utc 5915880 5241270 4400...
1,1,2021-11-21 14:22:26 SE Asia Standard Time,2021-11-21,14:22:26,bitcoingraph,btcusdt h4
2,2,2021-11-21 14:17:14 SE Asia Standard Time,2021-11-21,14:17:14,shanaz3197,tunefmofficial jam hbar
4,4,2021-11-21 14:07:27 SE Asia Standard Time,2021-11-21,14:07:27,khaiyaraya,first meeting crypto trading kepada yang ing...
5,5,2021-11-21 14:00:01 SE Asia Standard Time,2021-11-21,14:00:01,nowbitcoinprice,nov 21 2021 065900 utc 5927870 5251900 4408...
...,...,...,...,...,...,...
5784,5784,2021-11-21 14:49:42 SE Asia Standard Time,2021-11-21,14:49:42,stevenjhsiahaan,halo sekarang kamu bisa investasi saham luar n...
5785,5785,2021-11-21 14:45:28 SE Asia Standard Time,2021-11-21,14:45:28,samuelcendikia1,halo sekarang kamu bisa investasi saham luar n...
5786,5786,2021-11-21 14:39:45 SE Asia Standard Time,2021-11-21,14:39:45,saturnusgo,binnybunny96 bigalphaid kripto haram mbak ttd...
5787,5787,2021-11-21 14:35:25 SE Asia Standard Time,2021-11-21,14:35:25,muhamad25615484,halo sekarang kamu bisa investasi saham luar n...


In [48]:
data.isna().sum()

Unnamed: 0    0
created_at    0
date          0
time          0
username      0
tweet         0
dtype: int64

In [49]:
data = data.drop_duplicates(subset=['tweet'])
data

,Unnamed: 0,created_at,date,time,username,tweet
0,0,2021-11-21 14:30:01 SE Asia Standard Time,2021-11-21,14:30:01,nowbitcoinprice,nov 21 2021 072900 utc 5915880 5241270 4400...
1,1,2021-11-21 14:22:26 SE Asia Standard Time,2021-11-21,14:22:26,bitcoingraph,btcusdt h4
2,2,2021-11-21 14:17:14 SE Asia Standard Time,2021-11-21,14:17:14,shanaz3197,tunefmofficial jam hbar
4,4,2021-11-21 14:07:27 SE Asia Standard Time,2021-11-21,14:07:27,khaiyaraya,first meeting crypto trading kepada yang ing...
5,5,2021-11-21 14:00:01 SE Asia Standard Time,2021-11-21,14:00:01,nowbitcoinprice,nov 21 2021 065900 utc 5927870 5251900 4408...
...,...,...,...,...,...,...
5784,5784,2021-11-21 14:49:42 SE Asia Standard Time,2021-11-21,14:49:42,stevenjhsiahaan,halo sekarang kamu bisa investasi saham luar n...
5785,5785,2021-11-21 14:45:28 SE Asia Standard Time,2021-11-21,14:45:28,samuelcendikia1,halo sekarang kamu bisa investasi saham luar n...
5786,5786,2021-11-21 14:39:45 SE Asia Standard Time,2021-11-21,14:39:45,saturnusgo,binnybunny96 bigalphaid kripto haram mbak ttd...
5787,5787,2021-11-21 14:35:25 SE Asia Standard Time,2021-11-21,14:35:25,muhamad25615484,halo sekarang kamu bisa investasi saham luar n...


In [51]:
data.to_csv ('data.csv', index = None, header=True) 